# Fill in gaps in a time table 

In [6]:
import pandas as pd 


data = pd.read_csv("calls_server_across_time.csv").drop(["Unnamed: 0"], axis = 1)
data

,Time,Calls
0,2020-08-16 06:08:00,1.0
1,2020-08-16 06:09:00,1.0
2,2020-08-17 09:46:00,1.0
3,2020-08-17 09:47:00,1.0
4,2020-08-17 09:50:00,1.0
5,2020-08-17 09:53:00,1.0
6,2020-08-17 09:54:00,1.0
7,2020-08-17 12:06:00,1.0
8,2020-08-17 13:37:00,1.0
9,2020-08-17 13:38:00,1.0


In [7]:
df_org = data.copy(deep=True)
df_org['TimeGenerated'] = df_org['Time'].astype(object)

# Step 1: Convert Time to datetime object 
df_org.TimeGenerated = pd.to_datetime(df_org.TimeGenerated, format="%Y/%m/%d %H:%M:%S")

# Step 2: Create an index that goes from your minimum time to your maximum time and create a time 
# at the frequency you want. In our case, we want a new data point every minute => fre='min'
idx = pd.period_range(min(df_org.TimeGenerated), max(df_org.TimeGenerated), freq='min').strftime('%Y/%m/%d %H:%M:%S')
idx = pd.to_datetime(idx)

# Step 3: Build a dataframe based on this index 
df_date = pd.DataFrame()
df_date['TimeGenerated'] = idx

# Step 4: Merge this dataframe to your original dataset. If there is no point in your original dataset then
# you should expect to have an NA value in the merge. Those points are exactly the points where we have no failed 
# calls logged. Therefore we will NA values (~not failed calls) by a value of 0. 
df_merge = pd.merge(left = df_org, right = df_date, on='TimeGenerated', how = 'right').fillna(0)

df_merge = df_merge.loc[:, ['TimeGenerated', 'Calls']]
df_merge

,TimeGenerated,Calls
0,2020-08-16 06:08:00,1.0
1,2020-08-16 06:09:00,1.0
2,2020-08-16 06:10:00,0.0
3,2020-08-16 06:11:00,0.0
4,2020-08-16 06:12:00,0.0
...,...,...
1961,2020-08-17 14:49:00,0.0
1962,2020-08-17 14:50:00,0.0
1963,2020-08-17 14:51:00,0.0
1964,2020-08-17 14:52:00,0.0


In [8]:
# Save the above in a csv file 
df_merge.to_csv("calls_server_across_time_filled.csv", sep=",")